# 임베딩 모델 성능 비교

### 비교 기준

1. **임베딩 차원**: 벡터 크기 (차원이 높을수록 표현력 ↑, 메모리 ↑, 연산 속도 ⬇)
2. **비용**: API 비용 vs 로컬 서빙 비용
3. **메모리**: GPU 메모리 사용량 (로컬 모델)
4. **품질**: 의미적 유사도 정확도 (주관적, 벤치마크 기준)


In [1]:
# Huggingface Text Embedding Inference(TEI) 활용을 위한 공통 코드
# Dense Embedding, Reranker
import json
from collections.abc import Sequence

import httpx
from langchain_core.callbacks import Callbacks
from langchain_core.documents import BaseDocumentCompressor, Document
from langchain_core.embeddings import Embeddings


class HuggingfaceEmbedding(Embeddings):
    def __init__(self, url: str, model_kwargs: dict | None = None) -> None:
        self.url = url
        self.model_kwargs = model_kwargs

    def health_check(self) -> bool:
        try:
            httpx.get(f"{self.url}openapi.json")
            return True
        except Exception:
            return False

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """Call out to HuggingFaceHub's embedding endpoint for embedding search docs.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        # replace newlines, which can negatively affect performance.
        texts = [text.replace("\n", " ") for text in texts]
        _model_kwargs = self.model_kwargs or {}
        # NOTE: [API 문서](https://huggingface.github.io/text-embeddings-inference/#/Text%20Embeddings%20Inference/embed)
        responses = httpx.post(url=f"{self.url}embed", json={"inputs": texts, **_model_kwargs})
        responses.raise_for_status()
        return responses.json()

    async def aembed_documents(self, texts: list[str]) -> list[list[float]]:
        """Async Call to HuggingFaceHub's embedding endpoint for embedding search docs.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        # replace newlines, which can negatively affect performance.
        texts = [text.replace("\n", " ") for text in texts]
        _model_kwargs = self.model_kwargs or {}
        httpx_client = httpx.AsyncClient()
        responses = await httpx_client.post(
            url=f"{self.url}embed", json={"inputs": texts, "parameters": _model_kwargs}
        )
        return json.loads(responses.decode())

    def embed_query(self, text: str) -> list[float]:
        """Call out to HuggingFaceHub's embedding endpoint for embedding query text.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        response = self.embed_documents([text])[0]
        return response

    async def aembed_query(self, text: str) -> list[float]:
        """Async Call to HuggingFaceHub's embedding endpoint for embedding query text.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        response = (await self.aembed_documents([text]))[0]
        return response


# 위 HuggingfaceEmbedding 과 똑같이 TEI 서버를 호출하지만 Endpoint 가 다릅니다.
# https://huggingface.co/naver/xprovence-reranker-bgem3-v1
class HuggingfaceRerank(BaseDocumentCompressor):
    """Document compressor using a custom rerank service."""

    url: str = ""
    """URL of the custom rerank service."""
    top_n: int = 3
    """Number of documents to return."""
    batch_size: int = 10
    """Batch size to use for reranking."""

    def health_check(self) -> bool:
        try:
            httpx.get(f"{self.url}openapi.json")
            return True
        except Exception as e:
            print(f"Rerank 모델 서버 연결 실패: {str(e)}")
            return False

    def rerank(self, query: str, texts: list[str]) -> list[dict]:
        response = httpx.post(
            f"{self.url}rerank", json={"query": query, "texts": texts, "truncate": True}
        )
        if response.status_code != 200:
            raise RuntimeError(f"Failed to rerank documents, detail: {response}")
        return response.json()

    def compress_documents(
        self,
        documents: Sequence[Document],
        query: str,
        callbacks: Callbacks | None = None,
    ) -> Sequence[Document]:
        if not documents:
            print("No documents to compress")
            return []

        texts = [doc.page_content for doc in documents]
        batches = [texts[i : i + self.batch_size] for i in range(0, len(texts), self.batch_size)]
        all_results = []

        for batch in batches:
            results = self.rerank(query=query, texts=batch)
            all_results.extend(results)

        # Sort results based on scores and select top_n
        all_results = sorted(all_results, key=lambda x: x["score"], reverse=True)[: self.top_n]

        final_results = []
        for result in all_results:
            index = int(result["index"])
            metadata = documents[index].metadata.copy()
            metadata["relevance_score"] = result["score"]
            final_results.append(
                Document(page_content=documents[index].page_content, metadata=metadata)
            )

        return final_results

#### KURE-v1 Embedding via TEI(Based on BGE-M3)

1. TEI 서버 와 모델을 RunPod 에 셋팅합니다. 
- 직접 Docker Image 를 RunPod Template 으로 셋팅해야합니다.
  > 특히 HTTP Service PORT 셋팅에 주의하도록 해야합니다 + Environment Variables 에 필요한 값(MODEL_ID, HOST, PORT 등) 들을 셋팅합니다. 
- 실행 후에 바로 HTTP Service 에 접근하면 에러가 나는게 당연합니다. Model Download 및 Loading 까지는 시간이 소요됩니다.  
   Logs 로 살펴보겠습니다.


2. 해당 주소로 정상적인 접근이 되는지 확인합니다. (https://주소:8000/)

In [2]:
# TODO: TEI 서버 셋팅하고 그 주소에 맞게 셋팅이 필요합니다.
URL = "https://9v0dqiwjovw8zw-8000.proxy.runpod.net/"
bge_embedder = HuggingfaceEmbedding(url=URL)

test_text = "이것은 테스트 문장입니다."

# 테스트 임베딩
test_embedding_bge = bge_embedder.embed_query(test_text)
print(f"\n임베딩 차원 확인: {len(test_embedding_bge)}")

sample_embeddings_bge = bge_embedder.embed_documents(
    [
        "샘플 텍스트 테스트 중입니다.",
        "어떤 긴 글을 복사해서 임베딩 테스트를 해볼까요?",
    ]
)

print(sample_embeddings_bge)
print(f"{len(sample_embeddings_bge)}개 청크 임베딩 완료")

HTTPStatusError: Client error '404 Not Found' for url 'https://9v0dqiwjovw8zw-8000.proxy.runpod.net/embed'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404

#### 다양한 Dense Embedding Model: `Qwen3-Embedding-4B` via TEI

In [ ]:
# ============================================================================
# Qwen3-Embedding 모델 시리즈 활용하기
# ============================================================================

# NOTE: Qwen3-Embedding-0.6B 모델은 임베딩 차원을 1024 까지 지원합니다
# Qwen3-Embedding-4B 모델은 임베딩 차원을 2560 까지 지원합니다
# Qwen3-Embedding-8B 모델은 임베딩 차원을 4096 까지 지원합니다

# NOTE: MODEL_ID = Qwen/Qwen3-Embedding-4B
# NOTE: MAX_BATCH_TOKENS=40960
# TODO: TEI 서버(RUNPOD) 주소 - 실습 환경에 맞게 교체가 필요합니다.
RUNPOD_URL = "https://9v0dqiwjovw8zw-8000.proxy.runpod.net/"

try:
    qwen_embedder = HuggingfaceEmbedding(url=RUNPOD_URL)

    # 테스트 임베딩
    test_embedding_qwen = qwen_embedder.embed_query("임베딩 차원 테스트")
    print(test_embedding_qwen)
    print(f"   임베딩 차원: {len(test_embedding_qwen)}")

    sample_embeddings_qwen = qwen_embedder.embed_documents(["메롱", "메롱2"])

    print(f"   {len(sample_embeddings_qwen)}개 청크 임베딩 완료")

except Exception as e:
    print(f"\nQwen3 임베딩 초기화 실패: {e}")


Qwen3 임베딩 초기화 실패: Client error '404 Not Found' for url 'https://9v0dqiwjovw8zw-8000.proxy.runpod.net/embed'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/404


#### Qwen3-Reranker-0.6B via TEI

In [ ]:
# ============================================================================
# Qwen3-Reranker 모델 시리즈 활용하기
# ============================================================================

# NOTE: Qwen3-Reranker-0.6B 모델은 Context Window 32K 까지 지원합니다.

# NOTE: MODEL_ID = Qwen/Qwen3-Reranker-0.6B
# NOTE: MAX_BATCH_TOKENS=32768
# TODO: TEI 서버(RUNPOD) 주소 - 실습 환경에 맞게 교체가 필요합니다.
RUNPOD_URL = "https://9v0dqiwjovw8zw-8000.proxy.runpod.net/"

try:
    qwen_embedder = HuggingfaceEmbedding(url=RUNPOD_URL)

    # 테스트 임베딩
    test_embedding_qwen = qwen_embedder.embed_query("임베딩 차원 테스트")
    print(test_embedding_qwen)
    print(f"   임베딩 차원: {len(test_embedding_qwen)}")

    start = time.time()
    sample_embeddings_qwen = qwen_embedder.embed_documents(["메롱", "메롱2"])
    elapsed = time.time() - start

    print(f"   {len(sample_embeddings_qwen)}개 청크 임베딩 완료")
    print(f"   소요 시간: {elapsed:.2f}초")

except Exception as e:
    print(f"\nQwen3 임베딩 초기화 실패: {e}")